<a href="https://colab.research.google.com/github/soosoo22/Text-Summarization/blob/main/%08spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#!pip3 install newspaper3k

In [ ]:
#!pip3 install spaCy

In [ ]:
#!python3 -m spacy download en_core_web_sm

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

In [70]:
from newspaper import Article

url = 'https://sports.news.naver.com/news?oid=109&aid=0004843568'
article = Article(url)
article.download()
article.parse()

In [71]:
article.text

'[사진] 울산현대 제공. [사진] 울산현대 제공.\n\n[사진] 울산 현대 제공. [사진] 울산 현대 제공.\n\n[사진] 울산 현대 제공. [사진] 울산 현대 제공.\n\n[OSEN=고성환 기자] 울산현대가 어린이날 대구 원정을 떠난다.울산은 5일 오후 2시 DGB대구은행파크에서 대구FC와 하나원큐 K리그1 2023 11라운드 원정 경기를 치른다.현재 10경기를 소화한 울산은 8승 1무 1패 승점 25점으로 2위 그룹(FC서울, 포항스틸러스 승점19)과 격차를 6점으로 벌리며 단독 선두를 질주하고 있다.지난달 29, 30일에 치러졌던 K리그1 10라운드에서 울산은 홈 팀 중에 유일하게 승리를 챙겼다. 30일 호랑이굴에서 승격팀 광주FC를 맞아 후반 36분 선제골을 허용했지만, 41분 바코와 45분 주민규의 연속골에 힘입어 2-1로 짜릿한 역전승을 거뒀다.울산은 이번 시즌 ‘역전의 명수’다. 2월 25일 전북현대와 홈 개막전, 3월 12일 서울 원정, 4월 30일 광주전 모두 선제 실점을 한 뒤 고도의 집중력과 투혼, 결정력을 발휘하며 세 경기 모두 2-1 역전승을 챙겼다. 4월 22일 포항과 동해안더비에서는 두 골을 내준 뒤 두 골을 따라붙는 무서운 뒷심을 발휘, 2-2로 값진 승점 1점을 손에 넣었다.최근 팬들의 성원도 울산 상승세에 힘을 보태고 있다. 광주전은 일요일 저녁 경기임에도 1만 2,068명이 들어찼다. 실점한 순간에 팬들은 선수들처럼 전혀 흔들림 없었고, 마치 A매치 분위기를 방불케 하듯 모두 일어서서 함성과 박수로 열띤 응원을 보냈다. 그 결과 드라마 같은 기적을 만들 수 있었다. 2, 3월 K리그 최다 관중과 평균 관중 1위를 차지했던 울산은 홈 다섯 경기에서 총 8만 7,279명에 입장했다. 평균 1만 7,456명으로 평균 1만 관중 목표를 향해 순항 중이다.다시 2연승으로 불이 붙은 울산은 달구벌에서 3연승에 도전한다. 울산은 대구에 통산 전적에서 29승 14무 8패로 우위를 점하고 있다. 홍명보 감독 부임 후 4승 1무 2패를 기록 

In [72]:
def summarize(text, per):
  nlp = spacy.load('en_core_web_sm')
  doc = nlp(text)
  tokens = [token.text for token in doc]
  word_frequencies={}
  for word in doc:
    if word.text.lower() not in list(STOP_WORDS):
      if word.text.lower() not in punctuation:
        if word.text not in word_frequencies.keys():
          word_frequencies[word.text] = 1
        else:
          word_frequencies[word.text] += 1
  max_frequency = max(word_frequencies.values())
  for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequency
  sentence_tokens = [sent for sent in doc.sents]
  sentence_scores = {}
  for sent in sentence_tokens:
    for word in sent:
      if word.text.lower() in word_frequencies.keys():
        if sent not in sentence_scores.keys():
          sentence_scores[sent] = word_frequencies[word.text.lower()]
        else:
          sentence_scores[sent]+=word_frequencies[word.text.lower()]
  select_length = int(len(sentence_tokens)*per)
  summary=nlargest(select_length, sentence_scores, key=sentence_scores.get)
  final_summary=[word.text for word in summary]
  summary=''.join(final_summary)
  return summary


In [74]:
summarized_text = summarize(article.text, 0.05)
print('5% 요약 결과')
summarized_text

5% 요약 결과


'2월 25일 전북현대와 홈 개막전, 3월 12일 서울 원정, 4월 30일 광주전 모두 선제 실점을 한 뒤 고도의 집중력과 투혼, 결정력을 발휘하며 세 경기 모두 2-1 역전승을 챙겼다.30일 호랑이굴에서 승격팀 광주FC를 맞아 후반 36분 선제골을 허용했지만, 41분 바코와 45분 주민규의 연속골에 힘입어 2-1로 짜릿한 역전승을 거뒀다.울산은 이번 시즌 ‘역전의 명수’다.'